<a href="https://colab.research.google.com/github/Pavel184/NLP_basic_course/blob/Homework_lesson_9/Homework_lesson_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### HOMEWORK_LESSON_9

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

Mounted at /content/gdrive
gdrive	sample_data


In [3]:
%cd gdrive/My\ Drive/Colab Notebooks/NLP_basic_course/datasets

/content/gdrive/My Drive/Colab Notebooks/NLP_basic_course/datasets


In [4]:
!ls

 data_lesson_8		        text_3.txt
 pretrain		        text.txt
 ru_syntagrus-ud-dev.conllu     training_checkpoints
 ru_syntagrus-ud-train.conllu  'отзывы за лето.xls'
 text_2.txt


In [5]:
path_to_file = 'text_3.txt' # Устав ВС 

In [6]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 489696 characters


In [7]:
print(text[:500])

Настоящий Устав определяет права и обязанности военнослужащих Вооруженных Сил Россий-ской Федерации и взаимоотношения между ними, обязанности основных должностных лиц полка и его подразделений, а также правила внутреннего порядка.
	Настоящим Уставом руководствуются военнослужащие органов военного управления, во-инских частей, кораблей, предприятий, организаций Вооруженных Сил Российской Федерации, в том числе военных образовательных учреждений профессионального образования Министерства обороны 


In [8]:
text = text + text

In [9]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

95 unique characters


In [10]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [11]:
text_as_int, text, len(text_as_int), len(text)

(array([43, 61, 78, ...,  1,  2,  1]),
 'Настоящий Устав определяет права и обязанности военнослужащих Вооруженных Сил Россий-ской Федерации и взаимоотношения между ними, обязанности основных должностных лиц полка и его подразделений, а также правила внутреннего порядка.\r\n\tНастоящим Уставом руководствуются военнослужащие органов военного управления, во-инских частей, кораблей, предприятий, организаций Вооруженных Сил Российской Федерации, в том числе военных образовательных учреждений профессионального образования Министерства обороны Российской Федерации (далее - воинские части), и лица гражданского персонала, заме-щающие воинские должности. Положения Устава, в том числе обязанности основных должност-ных лиц полка и его подразделений, в равной степени относятся к военнослужащим всех воин-ских частей и подразделений.\r\n\tОбязанности должностных лиц, не указанных в Уставе, определяются соответствующими положениями, наставлениями, инструкциями и руководствами.\r\n\tДействие настоящег

### train and target

In [12]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

Н
а
с
т
о


In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Настоящий Устав определяет права и обязанности военнослужащих Вооруженных Сил Россий-ской Федерации и'
' взаимоотношения между ними, обязанности основных должностных лиц полка и его подразделений, а также '
'правила внутреннего порядка.\r\n\tНастоящим Уставом руководствуются военнослужащие органов военного упра'
'вления, во-инских частей, кораблей, предприятий, организаций Вооруженных Сил Российской Федерации, в '
'том числе военных образовательных учреждений профессионального образования Министерства обороны Росси'


In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [15]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Настоящий Устав определяет права и обязанности военнослужащих Вооруженных Сил Россий-ской Федерации '
Target data: 'астоящий Устав определяет права и обязанности военнослужащих Вооруженных Сил Россий-ской Федерации и'


In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [19]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [ ]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [20]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 95) # (batch_size, sequence_length, vocab_size)


In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 128)           12160     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          4722688   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 95)            97375     
                                                                 
Total params: 30,010,335
Trainable params: 30,010,335
No

In [23]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 95), dtype=float32, numpy=
array([[ 6.16946954e-06,  4.37571828e-07,  1.63484549e-06, ...,
         9.50354479e-06,  4.90446837e-06,  1.04169785e-05],
       [ 1.42054496e-05, -4.13826410e-06,  1.91681011e-06, ...,
         3.22144333e-05, -2.61389323e-06,  2.61148252e-05],
       [ 1.12792613e-05, -1.69764517e-05, -4.11004385e-06, ...,
         5.72064491e-05, -2.60111938e-05,  5.42622838e-05],
       ...,
       [ 1.02613203e-03,  1.30849774e-03, -1.79941460e-04, ...,
        -1.03772420e-03, -2.23191106e-03,  1.08365458e-03],
       [ 1.10949168e-03,  1.26114802e-03, -2.51849502e-04, ...,
        -1.01763615e-03, -2.26120884e-03,  1.05566124e-03],
       [ 1.20021612e-03,  1.20577752e-03, -3.03085282e-04, ...,
        -9.81547870e-04, -2.29766686e-03,  1.00590254e-03]], dtype=float32)>

In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [25]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' уставом Вооруженных Сил Российской Федерации. Подчи-ненные (младшие по воинскому званию) приветству'

Next Char Predictions: 
 '4_ЧЪ_ЩСС64й№НчИVшшн3тА«Ц»Ж)ъхV)Ь«ТВ\n«кю\t5пй\rнъЧну№О;Э()ЖАДОIФыдЗ№ЪБфюзциэхш4М.»а,Iм0нБЯфК(вЭЫМЫТПэв\n'


## Train the model

In [26]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 95)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.553966


In [27]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [28]:
!rm -rf ./training_checkpoints

In [29]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [31]:
EPOCHS = 40

In [32]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
151/151 [==============================] - 119s 737ms/step - loss: 3.3293
Epoch 2/40
151/151 [==============================] - 117s 764ms/step - loss: 2.2799
Epoch 3/40
151/151 [==============================] - 115s 749ms/step - loss: 1.3101
Epoch 4/40
151/151 [==============================] - 116s 761ms/step - loss: 1.0161
Epoch 5/40
151/151 [==============================] - 115s 752ms/step - loss: 0.8636
Epoch 6/40
151/151 [==============================] - 116s 760ms/step - loss: 0.7504
Epoch 7/40
151/151 [==============================] - 116s 760ms/step - loss: 0.6575
Epoch 8/40
151/151 [==============================] - 114s 745ms/step - loss: 0.5800
Epoch 9/40
151/151 [==============================] - 116s 758ms/step - loss: 0.5160
Epoch 10/40
151/151 [==============================] - 115s 750ms/step - loss: 0.4641
Epoch 11/40
151/151 [==============================] - 116s 762ms/step - loss: 0.4213
Epoch 12/40
151/151 [==============================] - 115s 751

## Generate text

In [33]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_39'

In [40]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [41]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (1, None, 128)            12160     
                                                                 
 lstm_8 (LSTM)               (1, None, 1024)           4722688   
                                                                 
 lstm_9 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_10 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_11 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 dense_2 (Dense)             (1, None, 95)             97375     
                                                                 
Total params: 30,010,335
Trainable params: 30,010,335


In [47]:
def generate_text(model, start_string, temperature):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = temperature

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [43]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже объем сохранность;
	д) исполнения обязанностей дежурного по роте пропола-ности.
	21. Порядок медицинской помощи назначаются дежурный из офицеров или прапорщика, ответственного за их хране-ние). Патроны, в мытовой подготовки в полку. При этом пностных лиц, ответ-ствбахте.
	398. При исполнении Государственного гимна Российской Федерации передает Боевое знамя знаменщиком и ассистентами под непо-средственному начальнику.
Начальник инженерной службы полка
	128. Начальник медицинской службы полка


In [44]:
len(text_)

515

In [64]:
for i in range(2, 16):
  text_ = generate_text(model, start_string=u"Командир роты  ", temperature=i/10)
  print('\n','temperature = ', i/10)
  print(text_, '\n')


 temperature =  0.2
Командир роты  прапорщика (мич-мана); постоянно проводить с офицерами работу по воинскому воспитанию и совершенствованию профессиональной подготовки;
	руководить технической подготовкой личного состава полка, совместно с начальником личного состава подразделений воинского эшелона (команды), в том числе об отставших военнослужащих, и предъявлять им маршрутный лист;
	обеспечивать соблюдение санитарно-гигиенических правил в воинском эшелоне;
	по указанию военных комендантов железнодорожных (водных) участков и  


 temperature =  0.3
Командир роты  прапорщика (мич-мана); постоянно проводить с офицерами работу по воинскому воспитанию и совершенствованию профессиональной подготовки;
	руководить технической подготовкой личного состава полка, совместно проживания и бытового обслуживания, принимать другие меры по охране здоровья и медицинскому обеспечению военнослужащих;
	заботиться об улучшении торгово-бытового обслуживания подчиненных, содержащихся на гауптвахте;
	рабочая